In [71]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
path = requests.get('https://inshorts.com/en/read')

In [5]:
path

<Response [200]>

In [6]:
path.status_code

200

In [11]:
page = path.content

In [12]:
soup = BeautifulSoup(page, 'html.parser')

In [14]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE doctype html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<style>
    /* The Modal (background) */
    .modal_contact {
        display: none; /* Hidden by default */
        position: fixed; /* Stay in place */
        z-index: 8; /* Sit on top */
        left: 0;
        top: 0;
        width: 100%; /* Full width */
        height: 100%;
        overflow: auto; /* Enable scroll if needed */
        background-color: rgb(0,0,0); /* Fallback color */
        background-color: rgba(0,0,0,0.4); /* Black w/ opacity */
    }

    /* Modal Content/Box */
    .modal-content {
        background-color: #fefefe;
        margin: 15% auto;
        padding: 20px !important;
        padding-top: 0 !important;
        /* border: 1px solid #888; */
        text-align: center;
        position: relative;
        border-radius: 6px;
    }

    /* The Close Button */
    .close {
      left: 90%;
      color: #aaa;
      float: right;
      font-size: 28px;


In [104]:
df = pd.DataFrame(columns=['article_headline','article_content'])

In [44]:
card_stack = soup.find(class_='card-stack')

In [55]:
articles = card_stack.find_all(class_="news-card z-depth-1")

In [1]:
for article in articles:
    heading = article.find('span',{'itemprop':'headline'}).get_text()
    article_content = article.find('div',{'itemprop':'articleBody'}).get_text()
    full_article = {'article_headline':heading, 'article_content':article_content}
    df.append(full_article , ignore_index=True)

NameError: name 'articles' is not defined

In [111]:
df.head()

,article_headline,article_content
